In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fraudfinder - ML Pipeline

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/vertex_ai/06_formalization.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/vertex_ai/06_formalization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/vertex_ai/06_formalization.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[Fraudfinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the Fraudfinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

This notebook shows how to use Feature Store, Pipelines and Model Monitoring for building an end-to-end demo using both components defined in `google_cloud_pipeline_components` and custom components. 

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)

Steps performed in this notebook:

* Create a Vetex AI Pipeline to orchestrate and automate the ML workflow

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [1]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)


BUCKET_NAME          = "fraud-finder-lab-fraudfinder"
PROJECT              = "fraud-finder-lab"
REGION               = "us-central1"
ID                   = "8wc8m"
FEATURESTORE_ID      = "fraudfinder_8wc8m"
MODEL_NAME           = "ff_model"
ENDPOINT_NAME        = "ff_model_endpoint"
TRAINING_DS_SIZE     = "1000"



### Import libraries and define constants

#### Libraries
Next you will import the libraries needed for this notebook. 

Note that currently this notebook uses KFP SDK v1, whereas the environment includes KFP v2. As an interim solution, we will downlevel KFP and the Google Cloud Pipeline Components in order to use the v1 code here as-is. See the [KFP migration guide](https://www.kubeflow.org/docs/components/pipelines/v2/migration/) for more details of moving from v1 to v2. 

In [2]:
# General
import os

# Vertex Pipelines
from typing import NamedTuple
import kfp
from kfp import dsl, compiler

from google.cloud import aiplatform as vertex_ai
from google_cloud_pipeline_components.v1 import dataset, custom_job, endpoint
from google_cloud_pipeline_components.types import artifact_types

In [3]:
print("kfp version:", kfp.__version__)

kfp version: 2.7.0


#### Variables

In [ ]:
# Components variables
BASE_IMAGE = "python:3.10"
COMPONENTS_DIR = os.path.join(os.curdir, "pipelines", "components")
INGEST_FEATURE_STORE = f"{COMPONENTS_DIR}/ingest_feature_store_{ID}.yaml"
TRAIN_MODEL = f"{COMPONENTS_DIR}/train_model_{ID}.yaml"
EVALUATE = f"{COMPONENTS_DIR}/evaluate_{ID}.yaml"

# Pipeline variables
PIPELINE_NAME = f"fraud-finder-xgb-pipeline-{ID}"
PIPELINE_DIR = os.path.join(os.curdir, "pipelines")
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipelines"
PIPELINE_PACKAGE_PATH = f"{PIPELINE_DIR}/pipeline_{ID}.json"

# Feature Store component variables
BQ_DATASET = "tx"
READ_INSTANCES_TABLE = f"ground_truth_{ID}"
READ_INSTANCES_URI = f"bq://{PROJECT_ID}.{BQ_DATASET}.{READ_INSTANCES_TABLE}"

# Dataset component variables
DATASET_NAME = f"fraud_finder_dataset_{ID}"

# Training component variables
JOB_NAME = f"fraudfinder-train-xgb-{ID}"
MODEL_REGISTRY_NAME = MODEL_NAME
MODEL_NAME = f"{MODEL_NAME}_xgb_pipeline_{ID}"
CONTAINER_URI = "us-docker.pkg.dev/vertex-ai/training/xgboost-cpu.1-7:latest"
MODEL_SERVING_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-7:latest"
)
ARGS = ["--bucket", f"gs://{BUCKET_NAME}"]
IMAGE_REPOSITORY = f"fraudfinder-{ID}"
IMAGE_NAME = "dask-xgb-classificator"
IMAGE_TAG = "v1"
IMAGE_URI = f"us-central1-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REPOSITORY}/{IMAGE_NAME}:{IMAGE_TAG}"  # TODO: get it from config

# Evaluation component variables
METRICS_URI = f"gs://{BUCKET_NAME}/deliverables/metrics.json"
AVG_PR_THRESHOLD = 0.2
AVG_PR_CONDITION = "avg_pr_condition"

# Endpoint variables
ENDPOINT_NAME = f"{ENDPOINT_NAME}_xgb_pipeline_{ID}"

# service account
SERVICE_ACCOUNT = "949651794396-compute@developer.gserviceaccount.com"

# command for trainer
COMMAND = ["python3", "trainer.py"]

# resource
PERSISTENT_RESOURCE_ID = None
REPLICA_COUNT = 1
MACHINE_TYPE = "n1-standard-4"

#### Initialize the Vertex AI SDK
Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [5]:
# Vertex AI SDK
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

In [6]:
!gsutil ubla set on gs://{BUCKET_NAME}

Enabling Uniform bucket-level access for gs://fraud-finder-lab-fraudfinder...


#### Create directories 
Create a directory for you pipeline and pipeline components. 

In [6]:
!mkdir -p -m 777 $PIPELINE_DIR $COMPONENTS_DIR

### Create a end-to-end Pipeline and execute it on Vertex AI Pipelines.

We will build a pipeline that you will execute using [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction). Vertex AI Pipelines helps you to automate, monitor, and govern your ML systems by orchestrating your ML workflow in a serverless manner, and storing your workflow's artifacts using Vertex ML Metadata. Authoring ML Pipelines that run on Vertex AI pipelines can be done in two different ways:

* [Tensorflow Extended](https://www.tensorflow.org/tfx/guide)
* [Kubeflow Pipelines SDK](https://kubeflow-pipelines.readthedocs.io/en/1.8.13/)

Based on your preference you can choose between the two options. This notebook will only focus on Kubeflow Pipelines.

If you don't have familiarity in authoring pipelines in Vertex AI Pipelines, we suggest the following resources:
* [Introduction to Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)
* [Build a Pipeline in Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/build-pipeline)

### Define Custom Components for your pipeline

We will use a mix of prebuilt (Google Cloud Pipeline Components) and custom components in this notebook. The difference is:

* Prebuilt components are official [Google Cloud Pipeline Components](https://cloud.google.com/vertex-ai/docs/pipelines/components-introduction)(GCPC). The GCPC Library provides a set of prebuilt components that are production quality, consistent, performant, and easy to use in Vertex AI Pipelines.
* As you will build in the cell below, a data scientist or ML engineer typically authored the custom component. This means you have more control over the component (container) code. In this case, it's a Python-function-based component. You also have the option to build a component yourself by packaging code into a container.

In the following two cells, you will build two custom components:

    *Feature Store component.

    *Evaluation component.

#### Define feature store component

Notice that the component assumes that contains the entities-timestamps "query" is already created.

# Feature Store
Next you will build a custom component using the [KFP SDK](https://kubeflow-pipelines.readthedocs.io/en/1.8.13/). Here you will take a Python function and create a component out of it. This component will take features from the Vertex AI Feature Store and output them on Google Cloud Storage (GCS). 

In [38]:
@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=["google-cloud-aiplatform==1.71.0"],
)
def ingest_features_gcs(
    project_id: str,
    region: str,
    bucket_name: str,
    feature_store_id: str,
    read_instances_uri: str,
) -> str:
    # Libraries --------------------------------------------------------------------------------------------------------------------------
    from datetime import datetime
    import glob
    import urllib
    import json

    # Feature Store
    from google.cloud.aiplatform import Featurestore

    # Variables --------------------------------------------------------------------------------------------------------------------------
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    bucket = urllib.parse.urlsplit(bucket_name).netloc
    export_uri = (
        f"{bucket_name}/data/snapshots/{timestamp}"  # format as new gsfuse requires
    )
    export_uri_path = f"/gcs/{bucket}/data/snapshots/{timestamp}"
    customer_entity = "customer"
    terminal_entity = "terminal"
    serving_feature_ids = {customer_entity: ["*"], terminal_entity: ["*"]}

    print(timestamp)
    print(bucket)
    print(export_uri)
    print(export_uri_path)
    print(customer_entity)
    print(terminal_entity)
    print(serving_feature_ids)

    # Main -------------------------------------------------------------------------------------------------------------------------------

    ## Define the feature store resource path
    feature_store_resource_path = (
        f"projects/{project_id}/locations/{region}/featurestores/{feature_store_id}"
    )
    print("Feature Store: \t", feature_store_resource_path)

    ## Run batch job request
    try:
        ff_feature_store = Featurestore(feature_store_resource_path)
        ff_feature_store.batch_serve_to_gcs(
            gcs_destination_output_uri_prefix=export_uri,
            gcs_destination_type="csv",
            serving_feature_ids=serving_feature_ids,
            read_instances_uri=read_instances_uri,
            pass_through_fields=["tx_fraud", "tx_amount"],
        )
    except Exception as error:
        print(error)

    # Store metadata
    snapshot_pattern = f"{export_uri_path}/*.csv"
    snapshot_files = glob.glob(snapshot_pattern)
    snapshot_files_fmt = [p.replace("/gcs/", "gs://") for p in snapshot_files]
    snapshot_files_string = json.dumps(snapshot_files_fmt)

    print(snapshot_pattern)
    print(snapshot_files)
    print(snapshot_files_fmt)
    print(snapshot_files_string)

    return snapshot_files_string


compiler.Compiler().compile(ingest_features_gcs, INGEST_FEATURE_STORE)

In [ ]:
@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=[
        "gcsfs==2024.10.0",
        "numpy==1.26.4", 
        "pandas==2.2.3", 
        "scikit-learn==1.5.2", 
        "dask==2024.10.0", 
        "dask-ml==2024.4.4",
        "distributed==2024.10.0", 
        "xgboost==2.1.2",
        "google-cloud-pipeline-components==2.17.0",
        "google-cloud-aiplatform==1.71.0",
    ]
)
def train_model(
    project: str,
    dataset: dsl.Input[artifact_types.VertexDataset],
    bucket: str,
    trained_model: dsl.Output[artifact_types.VertexModel],
    test_ds: dsl.Output[dsl.Dataset],
):
    from typing import List
    from pathlib import Path
    from datetime import datetime, timezone
    import dask.dataframe as dask_df
    from dask_ml.model_selection import train_test_split
    from dask.distributed import LocalCluster, Client
    import xgboost as xgb
    from google.cloud import aiplatform as vertex_ai


    ## Read environmental variables
    def gcs_path_to_local_path(old_path: str) -> str:
        new_path = old_path.replace("gs://", "/gcs/")
        return new_path

    ## Training variables
    BUCKET_NAME = "fraud-finder-lab-fraudfinder"
    REGION = "us-central1"
    LABEL_COLUMN = "tx_fraud"
    FEAT_COLUMNS = [
        'customer_id_avg_amount_14day_window', 
        'customer_id_avg_amount_15min_window', 
        'customer_id_avg_amount_1day_window', 
        'customer_id_avg_amount_30min_window', 
        'customer_id_avg_amount_60min_window', 
        'customer_id_avg_amount_7day_window', 
        'customer_id_nb_tx_14day_window', 
        'customer_id_nb_tx_15min_window', 
        'customer_id_nb_tx_1day_window', 
        'customer_id_nb_tx_30min_window', 
        'customer_id_nb_tx_60min_window', 
        'customer_id_nb_tx_7day_window', 
        'terminal_id_avg_amount_15min_window', 
        'terminal_id_avg_amount_30min_window', 
        'terminal_id_avg_amount_60min_window', 
        'terminal_id_nb_tx_14day_window', 
        'terminal_id_nb_tx_15min_window', 
        'terminal_id_nb_tx_1day_window', 
        'terminal_id_nb_tx_30min_window', 
        'terminal_id_nb_tx_60min_window', 
        'terminal_id_nb_tx_7day_window', 
        'terminal_id_risk_14day_window', 
        'terminal_id_risk_1day_window', 
        'terminal_id_risk_7day_window', 
        'tx_amount'
    ]
    UNUSED_COLUMNS = ["timestamp", "entity_type_customer", "entity_type_terminal"]
    DATA_SCHEMA = {
    "timestamp" : "object",
    "tx_amount": "float64",
    "tx_fraud": "Int64",
    "entity_type_customer": "Int64",
    "customer_id_nb_tx_1day_window": "Int64",
    "customer_id_nb_tx_7day_window": "Int64",
    "customer_id_nb_tx_14day_window": "Int64",
    "customer_id_avg_amount_1day_window": "float64",
    "customer_id_avg_amount_7day_window": "float64",
    "customer_id_avg_amount_14day_window": "float64",
    "customer_id_nb_tx_15min_window": "Int64",
    "customer_id_avg_amount_15min_window": "float64",
    "customer_id_nb_tx_30min_window": "Int64",
    "customer_id_avg_amount_30min_window": "float64",
    "customer_id_nb_tx_60min_window": "Int64",
    "customer_id_avg_amount_60min_window": "float64",
    "entity_type_terminal": "Int64",
    "terminal_id_nb_tx_1day_window": "Int64",
    "terminal_id_nb_tx_7day_window": "Int64",
    "terminal_id_nb_tx_14day_window": "Int64",
    "terminal_id_risk_1day_window": "float64",
    "terminal_id_risk_7day_window": "float64",
    "terminal_id_risk_14day_window": "float64",
    "terminal_id_nb_tx_15min_window": "Int64",
    "terminal_id_avg_amount_15min_window": "float64",
    "terminal_id_nb_tx_30min_window": "Int64",
    "terminal_id_avg_amount_30min_window": "float64",
    "terminal_id_nb_tx_60min_window": "Int64",
    "terminal_id_avg_amount_60min_window": "float64"
    }
    N_PARTITIONS = 4

    vertex_ai.init(project=project, location=REGION, staging_bucket=bucket)

    # manually extract and split 
    dataset_id = dataset.metadata['resourceName'].split("/")[-1]
    print("dataset_id", dataset_id)
    dataset = vertex_ai.TabularDataset(dataset.metadata['resourceName'])
    dataset_uris = dataset.gca_resource.metadata['inputConfig']['gcsSource']['uri']
    dataset_uris = [gcs_path_to_local_path(dataset_uri) for dataset_uri in dataset_uris]
    ds_df = dask_df.read_csv(dataset_uris, dtype=DATA_SCHEMA)
    train_df, test_df = train_test_split(ds_df, test_size=0.2, shuffle=True)
    eval_df, test_df = train_test_split(test_df, test_size=0.5)
    TRAINING_DIR = (
        f"/gcs/{BUCKET_NAME}/aiplatform-custom-training-"
        f"{datetime.now(timezone.utc).strftime('%Y-%m-%d-%H:%M:%S.%f')}"
    )
    TRAINING_DATA_DIR = (
        f"{TRAINING_DIR}/dataset-{dataset_id}-tables-"
        f"{datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%S.%fZ')}"
    )
    TRAINING_DATA_PATH = f"{TRAINING_DATA_DIR}/training-0000*-of-0000{N_PARTITIONS}.csv"
    EVAL_DATA_PATH = f"{TRAINING_DATA_DIR}/validation-0000*-of-0000{N_PARTITIONS}.csv"
    TEST_DATA_PATH = f"{TRAINING_DATA_DIR}/test-0000*-of-0000{N_PARTITIONS}.csv"
    train_df.repartition(npartitions=N_PARTITIONS).to_csv(TRAINING_DATA_PATH)
    eval_df.repartition(npartitions=N_PARTITIONS).to_csv(EVAL_DATA_PATH)
    test_df.repartition(npartitions=N_PARTITIONS).to_csv(TEST_DATA_PATH)

    MODEL_DIR = f"{TRAINING_DIR}/model"
    MODEL_PATH = f"{MODEL_DIR}/model.bst"
    MODEL_REGISTRY_NAME = "ff_model"


    def resample(df: dask_df.DataFrame, replace: bool, frac: float = 1, random_state: int = 8) -> dask_df.DataFrame:
        shuffled_df = df.sample(frac=frac, replace=replace, random_state=random_state)
        return shuffled_df

    def preprocess(df: dask_df.DataFrame, drop_cols: List[str] = None) -> dask_df.DataFrame:
        if drop_cols:
            df = df.drop(columns=drop_cols)

        # Drop rows with NaN"s
        df = df.dropna()

        # Convert integer valued (numeric) columns to floating point
        numeric_columns = df.select_dtypes(["float32", "float64"]).columns
        numeric_format = {col:"float32" for col in numeric_columns}
        df = df.astype(numeric_format)

        return df

    
    # preprocessing
    preprocessed_train_df = preprocess(train_df, UNUSED_COLUMNS)
    preprocessed_test_df = preprocess(test_df, UNUSED_COLUMNS)
    
    # downsampling
    train_nfraud_df = preprocessed_train_df[preprocessed_train_df[LABEL_COLUMN]==0]
    train_fraud_df = preprocessed_train_df[preprocessed_train_df[LABEL_COLUMN]==1]
    train_nfraud_downsample = resample(
        train_nfraud_df,
        replace=True, 
        frac=len(train_fraud_df)/len(train_df)
    )
    ds_preprocessed_train_df = dask_df.concat([train_nfraud_downsample, train_fraud_df])
    
    # target, features split
    x_train = ds_preprocessed_train_df[FEAT_COLUMNS].values
    y_train = ds_preprocessed_train_df.loc[:, LABEL_COLUMN].astype(int).values
    x_true = preprocessed_test_df[FEAT_COLUMNS].values
    y_true = preprocessed_test_df.loc[:, LABEL_COLUMN].astype(int).values
    preprocessed_test_df.to_csv(test_ds.path)
    
    # train model
    cluster =  LocalCluster()
    client = Client(cluster)
    model = xgb.dask.DaskXGBClassifier(objective="reg:logistic", eval_metric="logloss")
    model.client = client
    model.fit(x_train, y_train, eval_set=[(x_true, y_true)])
    if not Path(MODEL_DIR).exists():
        Path(MODEL_DIR).mkdir(parents=True, exist_ok=True)
    model.save_model(MODEL_PATH)
    # upload model
    vertex_ai_model = vertex_ai.Model.upload_xgboost_model_file(
        xgboost_version="1.7",
        model_file_path=MODEL_PATH,
        display_name=MODEL_REGISTRY_NAME,
    )
    trained_model.uri = vertex_ai_model.uri
    trained_model.metadata["resourceName"] = vertex_ai_model.resource_name
    trained_model.metadata["path"] = MODEL_PATH
    print(trained_model.metadata)
    print(trained_model.path, trained_model.uri)


compiler.Compiler().compile(train_model, TRAIN_MODEL)

#### Define an evaluate custom component
Next you will build a custom component that will evaluate our XGBoost model. This component will output `avg_precision_score` so that it can be used downstream for validating the model before deployment. 

In [ ]:
from typing import NamedTuple


@dsl.component(
    base_image=BASE_IMAGE,
    packages_to_install=[
        "gcsfs==2024.10.0",
        "numpy==1.26.4", 
        "pandas==2.2.3", 
        "scikit-learn==1.5.2", 
        "dask==2024.10.0", 
        "dask-ml==2024.4.4",
        "distributed==2024.10.0", 
        "xgboost==2.1.2",
        "google-cloud-pipeline-components==2.17.0",
    ],
)
def evaluate_model(
    threshold: float,
    model_in: dsl.Input[artifact_types.VertexModel],
    test_ds: dsl.Input[dsl.Dataset],
    metrics_uri: str,
) -> NamedTuple(
    "outputs",
    meta_metrics=dsl.Metrics,
    graph_metrics=dsl.ClassificationMetrics,
    avg_prec=float,
):
    # Libraries --------------------------------------------------------------------------------------------------------------------------
    import json
    from typing import NamedTuple
    import dask.dataframe as dask_df
    import numpy as np
    import xgboost as xgb
    import dask.dataframe as dask_df
    from sklearn.metrics import (confusion_matrix, average_precision_score, f1_score, 
                                log_loss, precision_score, recall_score)


    LABEL_COLUMN = "tx_fraud"
    FEAT_COLUMNS = [
        'customer_id_avg_amount_14day_window', 
        'customer_id_avg_amount_15min_window', 
        'customer_id_avg_amount_1day_window', 
        'customer_id_avg_amount_30min_window', 
        'customer_id_avg_amount_60min_window', 
        'customer_id_avg_amount_7day_window', 
        'customer_id_nb_tx_14day_window', 
        'customer_id_nb_tx_15min_window', 
        'customer_id_nb_tx_1day_window', 
        'customer_id_nb_tx_30min_window', 
        'customer_id_nb_tx_60min_window', 
        'customer_id_nb_tx_7day_window', 
        'terminal_id_avg_amount_15min_window', 
        'terminal_id_avg_amount_30min_window', 
        'terminal_id_avg_amount_60min_window', 
        'terminal_id_nb_tx_14day_window', 
        'terminal_id_nb_tx_15min_window', 
        'terminal_id_nb_tx_1day_window', 
        'terminal_id_nb_tx_30min_window', 
        'terminal_id_nb_tx_60min_window', 
        'terminal_id_nb_tx_7day_window', 
        'terminal_id_risk_14day_window', 
        'terminal_id_risk_1day_window', 
        'terminal_id_risk_7day_window', 
        'tx_amount'
    ]
    DATA_SCHEMA = {
    "timestamp" : "object",
    "tx_amount": "float64",
    "tx_fraud": "Int64",
    "entity_type_customer": "Int64",
    "customer_id_nb_tx_1day_window": "Int64",
    "customer_id_nb_tx_7day_window": "Int64",
    "customer_id_nb_tx_14day_window": "Int64",
    "customer_id_avg_amount_1day_window": "float64",
    "customer_id_avg_amount_7day_window": "float64",
    "customer_id_avg_amount_14day_window": "float64",
    "customer_id_nb_tx_15min_window": "Int64",
    "customer_id_avg_amount_15min_window": "float64",
    "customer_id_nb_tx_30min_window": "Int64",
    "customer_id_avg_amount_30min_window": "float64",
    "customer_id_nb_tx_60min_window": "Int64",
    "customer_id_avg_amount_60min_window": "float64",
    "entity_type_terminal": "Int64",
    "terminal_id_nb_tx_1day_window": "Int64",
    "terminal_id_nb_tx_7day_window": "Int64",
    "terminal_id_nb_tx_14day_window": "Int64",
    "terminal_id_risk_1day_window": "float64",
    "terminal_id_risk_7day_window": "float64",
    "terminal_id_risk_14day_window": "float64",
    "terminal_id_nb_tx_15min_window": "Int64",
    "terminal_id_avg_amount_15min_window": "float64",
    "terminal_id_nb_tx_30min_window": "Int64",
    "terminal_id_avg_amount_30min_window": "float64",
    "terminal_id_nb_tx_60min_window": "Int64",
    "terminal_id_avg_amount_60min_window": "float64"
    }


    def evaluate_model_fn(model: xgb.Booster, x_true: np.ndarray, y_true: np.ndarray, threshold: float = 0.5) -> dict:
        #calculate metrics
        metrics={}
        x_true = xgb.DMatrix(x_true)
        y_score = model.predict(x_true)
        y_pred = np.where(y_score >= threshold, 1, 0)
        c_matrix = confusion_matrix(y_true, y_pred)
        
        avg_precision_score = round(average_precision_score(y_true, y_score), 3)
        f1 = round(f1_score(y_true, y_pred), 3)
        lg_loss = round(log_loss(y_true, y_pred), 3)
        prec_score = round(precision_score(y_true, y_pred), 3)
        rec_score = round(recall_score(y_true, y_pred), 3)
        
        metrics["confusion_matrix"] = c_matrix.tolist()
        metrics["avg_precision_score"] = avg_precision_score
        metrics["f1_score"] = f1
        metrics["log_loss"] = lg_loss
        metrics["precision_score"] = prec_score
        metrics["recall_score"] = rec_score
        
        return metrics


    # load the dataframe, dask save to path as folder, need to put wildcard
    print("eval", test_ds.path)
    print("eval", model_in.path)
    test_df = dask_df.read_csv(f"{test_ds.path}/*", dtype=DATA_SCHEMA)
    test_df = test_df.compute()
    model = xgb.Booster()
    model.load_model(model_in.metadata["path"])
    eval_metrics = evaluate_model_fn(model, test_df[FEAT_COLUMNS], test_df[LABEL_COLUMN], threshold=threshold)

    # Variables --------------------------------------------------------------------------------------------------------------------------
    metrics_path = metrics_uri.replace("gs://", "/gcs/")
    labels = ["not fraud", "fraud"]

    # Main -------------------------------------------------------------------------------------------------------------------------------
    with open(metrics_path, mode="w") as metrics_file:
        json.dump(eval_metrics, metrics_file, indent=2)

    ## metrics
    c_matrix = eval_metrics["confusion_matrix"]
    avg_precision_score = eval_metrics["avg_precision_score"]
    f1 = eval_metrics["f1_score"]
    lg_loss = eval_metrics["log_loss"]
    prec_score = eval_metrics["precision_score"]
    rec_score = eval_metrics["recall_score"]

    meta_metrics = dsl.Metrics()
    meta_metrics.log_metric("avg_precision_score", avg_precision_score)
    meta_metrics.log_metric("f1_score", f1)
    meta_metrics.log_metric("log_loss", lg_loss)
    meta_metrics.log_metric("precision_score", prec_score)
    meta_metrics.log_metric("recall_score", rec_score)
    graph_metrics = dsl.ClassificationMetrics()
    graph_metrics.log_confusion_matrix(labels, c_matrix)


    ## model metadata
    # model_out.metadata["framework"] = "xgb.dask"
    # model_out.metadata["algorithm"] = "DaskXGBClassifier"
    # model_out.metadata["type"] = "classification"
    print("metadata metrics", meta_metrics.metadata)
    print("graph metrics", graph_metrics.metadata)

    eval_output = NamedTuple(
        "outputs",
        meta_metrics=dsl.Metrics,
        graph_metrics=dsl.ClassificationMetrics,
        avg_prec=float, 
    )
    return eval_output(
        meta_metrics=meta_metrics,
        graph_metrics=graph_metrics,
        avg_prec=avg_precision_score,
    )


compiler.Compiler().compile(evaluate_model, EVALUATE)

#### Author your pipeline
Next you will author the pipeline using the KFP SDK. This pipeline consists of the following steps:

* Ingest features
* Create Vertex AI Dataset
* Train XGBoost model
* Evaluate model
* Condition
* Create endpoint
* Deploy model into endpoint

In [ ]:
@dsl.pipeline(
    pipeline_root=PIPELINE_ROOT,
    name=PIPELINE_NAME,
)
def pipeline(
    project_id: str = PROJECT_ID,
    region: str = REGION,
    bucket_name: str = f"gs://{BUCKET_NAME}",
    feature_store_id: str = FEATURESTORE_ID,
    read_instances_uri: str = READ_INSTANCES_URI,
    replica_count: int = 1,
    machine_type: str = "n1-standard-4",
    # train_split: float = 0.8,
    # test_split: float = 0.1,
    # val_split: float = 0.1,
    metrics_uri: str = METRICS_URI,
    thold: float = AVG_PR_THRESHOLD,
):
    # Ingest data from featurestore
    ingest_features_op = ingest_features_gcs(
        project_id=project_id,
        region=region,
        bucket_name=bucket_name,
        feature_store_id=feature_store_id,
        read_instances_uri=read_instances_uri,
    )

    # create dataset
    dataset_create_op = dataset.TabularDatasetCreateOp(
        display_name=DATASET_NAME,
        project=project_id,
        gcs_source=ingest_features_op.output,
    ).after(ingest_features_op)

    # custom training job component - script
    train_model_component = custom_job.create_custom_training_job_from_component(
        train_model,
        display_name=JOB_NAME,
        replica_count=REPLICA_COUNT,
        machine_type=MACHINE_TYPE,
        base_output_directory=f"gs://{BUCKET_NAME}",
        service_account=SERVICE_ACCOUNT,
        # persistent_resource_id=PERSISTENT_RESOURCE_ID,
    )
    train_model_op = train_model_component(
        project=project_id,
        dataset=dataset_create_op.outputs["dataset"],
        bucket=f"gs://{BUCKET_NAME}",
    ).after(dataset_create_op)

    # evaluate component
    evaluate_model_op = evaluate_model(
        model_in=train_model_op.outputs["trained_model"], 
        test_ds=train_model_op.outputs["test_ds"],
        metrics_uri=metrics_uri,
    ).after(train_model_op)

    # if threshold on avg_precision_score
    with dsl.If(
        evaluate_model_op.outputs["avg_prec"] > thold, name=AVG_PR_CONDITION
    ):
        # create endpoint
        create_endpoint_op = endpoint.EndpointCreateOp(
            display_name=ENDPOINT_NAME,
            project=project_id,
            location=REGION,
        ).after(evaluate_model_op)

        # deploy the model
        custom_model_deploy_op = endpoint.ModelDeployOp(
            model=train_model_op.outputs["trained_model"],
            endpoint=create_endpoint_op.outputs["endpoint"],
            deployed_model_display_name=MODEL_NAME,
            dedicated_resources_machine_type=machine_type,
            dedicated_resources_min_replica_count=replica_count,
        ).after(create_endpoint_op)

1


#### Compile and run the pipeline
After authoring the pipeline you can use the compiler to compile the pipeline. 

In [48]:
# compile the pipeline
pipeline_compiler = compiler.Compiler()
pipeline_compiler.compile(pipeline_func=pipeline, package_path=PIPELINE_PACKAGE_PATH)

Next you can use the Vertex AI SDK to create a job on Vertex AI Pipelines. 

In [ ]:
# instantiate pipeline representation
pipeline_job = vertex_ai.PipelineJob(
    display_name=PIPELINE_NAME,
    template_path=PIPELINE_PACKAGE_PATH,
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
)

In [ ]:
# submit the pipeline run (may take ~20 minutes for the first run)
pipeline_job.run(sync=True)

In [ ]:
x1 = artifact_types.VertexDataset()

x1.uri

custom_job.CustomTrainingJobOp

from google_cloud_pipeline_components.v1.automl import training_job

training_job.AutoMLTabularTrainingJobRunOp

from google.cloud.aiplatform import AutoMLTabularTrainingJob

from kfp.dsl.pipeline_channel import PipelineArtifactChannel

''

### END

Now you can go to the next notebook `07_deployment.ipynb` and explore deployment using Cloud Build